In [1]:
import os, requests
from io import BytesIO
from zipfile import ZipFile
from glob import glob
import numpy as np
np.random.seed(1775)

In [3]:
DATA_DIR = '/data/rgbd_face_data/kinect'
pjoin = os.path.join
os.makedirs(DATA_DIR)
os.chdir(DATA_DIR)

In [4]:
links=["http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-16)(151751).zip",
       "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-16)(153054).zip",
       "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-16)(154211).zip",
       "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-16)(160440).zip",
       "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-16)(160931).zip",
       "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-16)(161342).zip",
       "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-16)(163349).zip",
       "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-16)(164248).zip",
       "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-17)(141550).zip",
       "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-17)(142154).zip",
       "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-17)(142457).zip",
       "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-17)(143016).zip",
       "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-18)(132824).zip", 
       "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-18)(133201).zip", 
       "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-18)(133846).zip",
       "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-18)(134239).zip", 
       "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-18)(134757).zip", 
       "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-18)(140516).zip", 
       "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-18)(143345).zip", 
       "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-18)(144316).zip", 
       "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-18)(145150).zip", 
       "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-18)(145623).zip", 
       "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-18)(150303).zip",
       "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-18)(150650).zip", 
       "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-18)(151337).zip", 
       "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-18)(151650).zip",
       "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-18)(152717).zip", 
       "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-18)(153532).zip", 
       "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-18)(154129).zip", 
       "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-18)(154728).zip", 
       "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-18)(155357).zip"]

In [5]:
for i, link in enumerate(links):
    r = requests.get(link, stream=True)
    with ZipFile(BytesIO(r.content)) as z:
        z.extractall()
    os.rename(os.path.basename(link[:-4]), "s"+str(i+1))
    print('{}% complete'.format(((i+1)/len(links))*100))

0.0% complete
3.225806451612903% complete
6.451612903225806% complete
9.67741935483871% complete
12.903225806451612% complete
16.129032258064516% complete
19.35483870967742% complete
22.58064516129032% complete
25.806451612903224% complete
29.03225806451613% complete
32.25806451612903% complete
35.483870967741936% complete
38.70967741935484% complete
41.935483870967744% complete
45.16129032258064% complete
48.38709677419355% complete
51.61290322580645% complete
54.83870967741935% complete
58.06451612903226% complete
61.29032258064516% complete
64.51612903225806% complete
67.74193548387096% complete
70.96774193548387% complete
74.19354838709677% complete
77.41935483870968% complete
80.64516129032258% complete
83.87096774193549% complete
87.09677419354838% complete
90.32258064516128% complete
93.54838709677419% complete
96.7741935483871% complete


In [6]:
test = np.random.choice(os.listdir(), size=len(links)//10).tolist()
train = list(set(os.listdir())-set(test))

In [7]:
os.makedirs(pjoin(DATA_DIR,'train'))
os.makedirs(pjoin(DATA_DIR, 'test'))

In [8]:
for folder in test:
    os.rename(folder, 'test/'+folder)
for folder in train:
    os.rename(folder, 'train/'+folder)

In [11]:
train_dir = '/data/rgbd_face_data/kinect/train'
test_dir = '/data/rgbd_face_data/kinect/test'
for old_dir in glob(train_dir + '/*') + glob(test_dir + '/*'):
    os.makedirs(old_dir.replace('kinect', 'kinect_cropped'))
    for item in glob.glob(old_dir + '/*'):
        if item.endswith('.dat'):
            dmap = pd.read_csv(item, sep='\t', header=None).values
            dmap = dmap[140:340, 220:420]
            dmap[(dmap > 3000)|(dmap == -1)] = 3000
            dmap = (dmap - np.mean(dmap))/np.max(dmap)
            dmap = np.uint8(dmap*255)
            dmap = Image.fromarray(dmap, 'L').resize((100,100))
            dmap = np.asarray(dmap)
            fname = item.replace('kinect', 'kinect_cropped')
            fname = fname.replace('.dat', '.npy')
            np.save(fname, dmap)
        else:
            img = Image.open(item)
            img = img.convert('RGB')
            img.thumbnail((640,480))
            img = np.asarray(img)[140:340, 220:420,:]
            img = Image.fromarray(img).resize((100,100))
            fname = item.replace('kinect', 'kinect_cropped')
            img.save(fname)
            _, hog_image = hog(img, orientations=8, pixels_per_cell=(2, 2),
                               cells_per_block=(1, 1), visualize=True,
                               multichannel=True, block_norm='L2',
                               transform_sqrt=True)
            fname = fname.replace('c.bmp', 'h.npy')
            np.save(fname,hog_image)

['/data/rgbd_face_data/kinect/train/s25',
 '/data/rgbd_face_data/kinect/train/s16',
 '/data/rgbd_face_data/kinect/train/s21',
 '/data/rgbd_face_data/kinect/train/s11',
 '/data/rgbd_face_data/kinect/train/s18',
 '/data/rgbd_face_data/kinect/train/s27',
 '/data/rgbd_face_data/kinect/train/s8',
 '/data/rgbd_face_data/kinect/train/s24',
 '/data/rgbd_face_data/kinect/train/s30',
 '/data/rgbd_face_data/kinect/train/s17',
 '/data/rgbd_face_data/kinect/train/s5',
 '/data/rgbd_face_data/kinect/train/s19',
 '/data/rgbd_face_data/kinect/train/s20',
 '/data/rgbd_face_data/kinect/train/s10',
 '/data/rgbd_face_data/kinect/train/s13',
 '/data/rgbd_face_data/kinect/train/s14',
 '/data/rgbd_face_data/kinect/train/s26',
 '/data/rgbd_face_data/kinect/train/s23',
 '/data/rgbd_face_data/kinect/train/s3',
 '/data/rgbd_face_data/kinect/train/s6',
 '/data/rgbd_face_data/kinect/train/s9',
 '/data/rgbd_face_data/kinect/train/s7',
 '/data/rgbd_face_data/kinect/train/s2',
 '/data/rgbd_face_data/kinect/train/s4',
